In [1]:
# AUDIT VALERIO
import os
from pyspark.sql import SparkSession
import time
from scipy.stats import gmean

# path to files
import glob

#librairies to read/tranforme/write draw data
import csv
import xml.etree.ElementTree as ET
import pandas as pd

# DIMESSAGES VALERIO
from pyspark.sql.functions import current_timestamp
from datetime import datetime

# Initialize SparkSession
spark = SparkSession.builder.appName("TPC-DS Data Loading").config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0").config("spark.sql.catalogImplementation", "hive").getOrCreate()

23/12/18 15:23:26 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.178.59 instead (on interface eno1)
23/12/18 15:23:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/coucou/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/coucou/.ivy2/cache
The jars for the packages stored in: /home/coucou/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-db426fcd-eb06-492d-9cfb-d445966229c9;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 386ms :: artifacts dl 10ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| 

Tags Valerio modification
1. "PERFORM UPDATE VALERIO" -> Update of Prospect table in Incremental Phases
2. "AUDIT VALERIO" -> Steps linked to perform the Audit Phase
3. "DIMESSAGES VALERIO" -> Steps linked to output DImessages

In [2]:
#Set up a set of variables
# SETUP the name of the file where result will be write
SF = 10 #choose between 1,3,5,8,10

#order of table to load

Not_modified_table = [["Time", "DimTime"], ["Date", "DimDate"],["Industry", "Industry"],
                     ["StatusType", "StatusType"],["TaxRate", "TaxRate"],["TradeType","TradeType"]]

FINWIRE_table = [["FINWIRE","DimCompany"],["FINWIRE","DimSecurity"],["FINWIRE","Financial"]]

Modified_DimTable = [["CustomerMgmt.csv","DimCustomer"],["HR.csv","DimBroker"],
                         ["CustomerMgmt.csv","DimAccount"], ["Trade.txt","DimTrade"]] 


Modified_FactTable = [["CashTransaction.txt","FactCashBalances"],["HoldingHistory.txt","FactHoldings"]
                                  ,["DailyMarket.txt","FactMarketHistory"],["WatchHistory.txt","FactWatches"]
                                  ,["Prospect.csv","Prospect"]]


#How split FINWIRE FILES

cmp_colspecs=[(0, 15), (15, 18), (18, 78), (78, 88), (88, 92), (92, 94), (94, 98), (98, 106), (106, 186), (186, 266), (266, 278), (278, 303), (303, 323), (323, 347), (347, 393), (393, 543)]
fin_colspecs=[(0, 15), (15, 18), (18, 22), (22, 23), (23, 31), (31, 39), (39, 56), (56, 73), (73, 85), (85, 97), (97, 109), (109, 126), (126, 143), (143, 160), (160, 173), (173, 186), (186, 246)]
sec_colspecs=[(0, 15), (15, 18), (18, 33), (33, 39), (39, 43), (43, 113), (113, 119), (119, 132), (132, 140), (140, 148), (148, 160), (160, 220)]



# DIMESSAGES VALERIO

valid_ratings = ["AAA", "AA[+/-]", "A[+/-]", "BBB[+/-]", "BB[+/-]", "B[+/-]", "CCC[+/-]", "CC", "C", "D"]


In [3]:
f = open(os.getcwd() + "/ddl/tpcdi.sql", "r")
tables=f.read()
tables = tables.split(";")[1:-1]
for i in tables:
    spark.sql(i)

# AUDIT VALERIO
# If not done: create the audit table in tpcdi.sql

23/12/18 15:24:20 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/12/18 15:24:21 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/12/18 15:24:21 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/12/18 15:24:42 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/12/18 15:24:42 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore coucou@127.0.1.1
23/12/18 15:24:42 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
23/12/18 15:24:44 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAu

23/12/18 15:24:48 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/12/18 15:24:48 WARN HiveMetaStore: Location: file:/home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/spark-warehouse/audit specified for non-external table:audit


In [ ]:
# AUDIT VALERIO
def visibility_2_fun():
    
    global start_time
    end_time = time.time()
    execution_time = end_time - start_time
    
    if execution_time > 300: # 300 seconds = 5 minutes
        print("ERROR: more than 5 minutes elapsed between visibility queries's executions")
        
    elif execution_time < 300 and execution_time > 120:
        spark.sql(os.getcwd() + "/ddl/tpcdi_visibility_2.sql")
        start_time = time.time()
        return
        
    else:
        return

In [10]:
# AUDIT VALERIO
# This function loads audit data into the audit table. It can be used for all the three batches.

def audit_upload(batch_n):
    # Go to Batch1 and consider only the files that end for "audit.csv"
    audit_files = glob.glob(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+ str(batch_n) +"/*_audit.csv")
    
    # For each audit_file...
    sch_audit = spark.sql("select * from audit").schema
    for file_path in audit_files:
        # Place "audit_file" data into a Spark DF
        audit_df = spark.read.schema(sch_audit).csv(file_path, header=True, inferSchema=True)

        # Append data to the Audit table
        audit_df.write.insertInto("Audit", overwrite=False)

In [6]:
# AUDIT VALERIO
# Just before startin the Historical Load: execute the batch validation query

#I propose to create a new folders for all files.sql ??

f = open(os.getcwd() + "/ddl/tpcdi_validation.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

#spark.sql(os.getcwd() + "/data/output/tpcdi_validation.sql")

-------------------------------
/* ++++++++++++++++++++++++++++++++++++++++++++++++++ *
 * +                                                + *
 * +        TPC-DI  Validation Query                + *
 * +        Version 1.1.0                           + *
 * +                                                + *
 * ++++++++++++++++++++++++++++++++++++++++++++++++++ *
 *                                                    *
 *       ====== Portability Substitutions ======      *
 *     ---        [FROM DUMMY_TABLE]    ------        *
 * DB2            from sysibm.sysdummy1               *
 * ORACLE         from dual                           *
 * SQLSERVER       <blank>                            *
 * -------------------------------------------------- *
 *     ------  [||] (String concatenation) ------     *    
 * SQLSERVER      +                                   *
 * -------------------------------------------------- *
 */

insert into DImessages

select

     CURRENT_TIMESTAMP as Messa

++
||
++
++

-------------------------------
 
/* Phase complete record */
insert into DImessages
select
     MessageDateAndTime
     ,case when BatchID is null then 0 else BatchID end as BatchID
     ,MessageSource
     ,MessageText 
     ,MessageType
     ,MessageData
from (
     select CURRENT_TIMESTAMP as MessageDateAndTime
            ,max(BatchID) as BatchID
            ,'Phase Complete Record' as MessageSource
            ,'Batch Complete' as MessageText
            ,'PCR' as MessageType
            ,NULL as MessageData
  from DImessages
) 

++
||
++
++



In [ ]:
#Not_modified_table
for i in Not_modified_table:
    name_file = glob.glob(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/"+ i[0] +".txt")
    df_schema = spark.sql("select * from " + i[1])
    df = spark.read.schema(df_schema.schema).option("delimiter", "|").csv(name_file)
    df.write.mode("overwrite").insertInto(i[1])
            

In [ ]:
def string_to_date(string):
    return string[0:4]+"-"+string[4:6]+"-"+string[6:8]
    
def string_to_timestamp(string):
    return string[0:4]+"-"+string[4:6]+"-"+string[6:8]+" " +string[9:11]+":"+string[11:13]+":"+string[13:15]

In [ ]:
#finwire_files = [ finwire_source_files_path + '/' + file for file in os.listdir( finwire_source_files_path ) if re.search( r'^FINWIRE......$', file ) ] 
finwire_combined_df = pd.DataFrame()
finwire_files = glob.glob(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/FINWIRE??????")
finwire_files.sort()

#raise Exception("job done")

for file_path in finwire_files:
    #Read the file and add to a combined Dataframe
    c_names = ["Content"]
    finwire_df = pd.read_csv(file_path, names=c_names,index_col=False)
    finwire_df['Type'] = pd.DataFrame(finwire_df["Content"].str.slice(15, 18))
    finwire_combined_df = pd.concat([finwire_combined_df, finwire_df], ignore_index=True)
    finwire_df_CMP=finwire_combined_df[finwire_combined_df["Type"]=='CMP']
    finwire_df_FIN=finwire_combined_df[finwire_combined_df["Type"]=='FIN']
    finwire_df_SEC=finwire_combined_df[finwire_combined_df["Type"]=='SEC']
    finwire_df_CMP=pd.DataFrame(finwire_df_CMP[["Content"]])
    finwire_df_FIN=pd.DataFrame(finwire_df_FIN[["Content"]])
    finwire_df_SEC=pd.DataFrame(finwire_df_SEC[["Content"]])
    
    skiprows = 1
    serrogate_key_CMP = 0
    serrogate_key_SEC = 0
    
    
    if finwire_df_CMP.empty  == False:
        finwire_df_CMP.to_csv("CMP", index=False)
        df_CMP = pd.read_fwf("CMP" , colspecs=cmp_colspecs, header = None, skiprows=skiprows)
        
        csv_file = open(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_CMP.csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)
        for index, row in df_CMP.iterrows():
            #set value
            Industry = spark.sql("select IN_NAME from INDUSTRY where IN_ID = '" + row[5] + "'").collect()[0][0]
            Status = spark.sql("select ST_NAME from StatusType where ST_ID = '" + row[4] + "'").collect()[0][0]
            # VALERIO DIMESSAGES
            if row[6] not in valid_ratings:
                df = [(current_timestamp(),1,"DimCompany","Invalid SPRating","Alert",f"CO_ID = {row[3]}, CO_SP_RATE ={row[6]}")]
                df.write.mode("append").insertInto("DImessages")
                isLowGrade = None
                row[6] = None
                
            else:                
                if row[6].startswith('A') or row[6].startswith('BBB'):
                    isLowGrade = 0
                else:
                    isLowGrade = 1
            # ---
            
            #reorder
            working_df = [serrogate_key_CMP,row[3],Status,row[2],Industry,row[6],isLowGrade,row[14],row[8],row[9]
                          ,row[10],row[11],row[12],row[13],row[15],row[7],1,1
                          ,string_to_timestamp(row[0]),'9999-12-31']
            csv_writer.writerow(working_df)
            serrogate_key_CMP += 1

            
        sch = spark.sql("select * from DimCompany").schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_CMP.csv", sep=',')
        df.write.mode("append").insertInto("DimCompany")
         

    
    if finwire_df_SEC.empty == False:
        finwire_df_SEC.to_csv("SEC", index=False)
        df_SEC = pd.read_fwf("SEC" , colspecs=sec_colspecs, header = None, skiprows=skiprows)
        
        csv_file = open(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_SEC.csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)
        for index, row in df_SEC.iterrows():
            #set value
            SK_SECURITYID = spark.sql("""select SK_CompanyID from DimCompany
                        where (Name == '"""+ str(row[11]) +"""' or CompanyID== '""" 
                      + str(row[11]) + """' ) and EffectiveDate <= '""" + string_to_timestamp(row[0]) +
                      """' and '"""+ string_to_timestamp(row[0]) + """' < EndDate""").collect()[0][0]
            print(row[4])
            Status = spark.sql("select ST_NAME from StatusType where ST_ID = '" + row[4] + "'").collect()[0][0]
            
            #reorder
            working_df = [serrogate_key_SEC, row[2], row[3], Status, row[5], row[6], SK_SECURITYID, row[7], row[8]
                         , row[9], row[10],1,1,string_to_timestamp(row[0]),'9999-12-31']
            csv_writer.writerow(working_df)
            serrogate_key_SEC += 1
        
        #add result
        sch = spark.sql("select * from DimSecurity").schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_SEC.csv", sep=',')
        df.write.mode("append").insertInto("DimSecurity")
        
    if finwire_df_FIN.empty == False:
        finwire_df_FIN.to_csv("FIN", index=False)
        df_FIN = pd.read_fwf("FIN" , colspecs=fin_colspecs, header = None, skiprows=skiprows)
        
        csv_file = open(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_FIN.csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)
        for index, row in df_FIN.iterrows():
            SK_COMPANYID =spark.sql("""select SK_CompanyID from DimCompany
                        where (Name == '"""+ str(row[16]) +"""' or CompanyID== '""" 
                      + str(row[16]) + """' ) and EffectiveDate <= '""" + string_to_timestamp(row[0]) +
                      """' and '"""+ string_to_timestamp(row[0]) + """' < EndDate""").collect()[0][0]
                    
            working_df = [SK_COMPANYID, row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]
                         , row[10], row[11], row[12], row[13], row[14], row[15]]
            csv_writer.writerow(working_df)
            
        sch = spark.sql("select * from Financial").schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/" + file_path.split('/')[-1] + "_FIN.csv", sep=',')
        df.write.mode("append").insertInto("Financial")
    

In [ ]:
## TRANSOFRMING customer.xml in customer.csv

In [ ]:
tree = ET.parse(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/CustomerMgmt.xml")
root = tree.getroot()

def finder(obj_name):
    try:
        return Customer.find(f".//{obj_name}").text
    except:
        return ""

# Create the .csv
print(os.getcwd() + "/CustomerMgmt.csv")
with open(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/CustomerMgmt.csv", 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    header = row = ['ActionType','ActionTS','C_ID', 'C_TAX_ID', 'C_GNDR', 'C_TIER', 'C_DOB', 'C_L_NAME', 'C_F_NAME', 'C_M_NAME', 'C_ADLINE1', 'C_ADLINE2', 'C_ZIPCODE', 'C_CITY', 'C_STATE_PROV', 'C_CTRY', 'C_PRIM_EMAIL', 'C_ALT_EMAIL', 'C_CTRY_CODE_1', 'C_AREA_CODE_1', 'C_LOCAL_1', 'C_EXT_1', 'C_CTRY_CODE_2', 'C_AREA_CODE_2', 'C_LOCAL_2', 'C_EXT_2', 'C_CTRY_CODE_3', 'C_AREA_CODE_3', 'C_LOCAL_3', 'C_EXT_3', 'C_LCL_TX_ID', 'C_NAT_TX_ID', 'CA_ID', 'CA_TAX_ST', 'CA_B_ID', 'CA_NAME']

    csv_writer.writerow(header)
    
    # Iteratively add the customers
    for TPCDAction in root:
        row = []

        action_type = TPCDAction.attrib.get("ActionType")
        action_ts = TPCDAction.attrib.get("ActionTS")

        for Customer in TPCDAction:
            c_id = Customer.attrib.get("C_ID")
            c_tax_id = Customer.attrib.get("C_TAX_ID")
            c_gndr = Customer.attrib.get("C_GNDR")
            c_tier = Customer.attrib.get("C_TIER")
            c_dob = Customer.attrib.get("C_DOB")
            
            c_l_name = finder("C_L_NAME")
            c_f_name = finder("C_F_NAME")
            c_m_name = finder("C_M_NAME")
            
            c_adline1 = finder("C_ADLINE1")
            c_adline2 = finder("C_ADLINE2")
            c_zipcode = finder("C_ZIPCODE")
            c_city = finder("C_CITY")
            c_state_prov = finder("C_STATE_PROV")
            c_ctry = finder("C_CTRY")
        
            c_prim_email = finder("C_PRIM_EMAIL")
            c_alt_email = finder("C_ALT_EMAIL")
            
            c_ctry_code_1 = finder("C_PHONE_1//C_CTRY_CODE")
            c_area_code_1 = finder("C_PHONE_1//C_AREA_CODE")
            c_local_1 = finder("C_PHONE_1//C_LOCAL")
            c_ext_1 = finder("C_PHONE_1//C_EXT")
        
            c_ctry_code_2 = finder("C_PHONE_2//C_CTRY_CODE")
            c_area_code_2 = finder("C_PHONE_2//C_AREA_CODE")
            c_local_2 = finder("C_PHONE_2//C_LOCAL")
            c_ext_2 = finder("C_PHONE_2//C_EXT")

            c_ctry_code_3 = finder("C_PHONE_3//C_CTRY_CODE")
            c_area_code_3 = finder("C_PHONE_3//C_AREA_CODE")
            c_local_3 = finder("C_PHONE_3//C_LOCAL")
            c_ext_3 = finder("C_PHONE_3//C_EXT")
            
            c_lcl_tx_id = finder(f"C_LCL_TX_ID")
            c_nat_tx_id = finder(f"C_NAT_TX_ID")
            
            try:
                ca_id = Customer.find(".//Account").attrib.get("CA_ID")
            except:
                ca_id = ""
            
            try:
                ca_tax_st = Customer.find(".//Account").attrib.get("CA_TAX_ST")
            except:
                ca_tax_st = ""
            
            ca_b_id = finder(f"CA_B_ID")
            ca_name = finder(f"CA_NAME")
        
        row = [action_type, action_ts, c_id, c_tax_id, c_gndr, c_tier, c_dob, c_l_name, c_f_name, c_m_name, c_adline1, c_adline2, c_zipcode, c_city, c_state_prov, c_ctry, c_prim_email, c_alt_email, c_ctry_code_1, c_area_code_1, c_local_1, c_ext_1, c_ctry_code_2, c_area_code_2, c_local_2, c_ext_2, c_ctry_code_3, c_area_code_3, c_local_3, c_ext_3, c_lcl_tx_id, c_nat_tx_id, ca_id, ca_tax_st, ca_b_id, ca_name]
        
        csv_writer.writerow(row)   

In [ ]:
## DIMENSION TABLE LOAD

In [ ]:
from pyspark.sql.functions import col, when, lit, trunc, upper, concat
#Modified_table
for i in Modified_DimTable:
    name_file = glob.glob(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/" +i[0])

    if i[0] == "HR.csv":
        csv_file = open(os.getcwd() + "/result/Batch1_" + i[1] +".csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)
        hr_df = pd.read_csv(name_file[0], header=None,sep=r'[,|]')
        broker_df = hr_df[hr_df[5]== 314] # Filter for brokers only
        SK_BrokerID = 0
        min_date = spark.sql("select min(DATEVALUE) from DimDate")# Get the earliest date in the DimDate table
        for index, row in broker_df.iterrows():
            working_df = [SK_BrokerID,row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],
                         'True', "9999-12-31",min_date, '1']
            SK_BrokerID +=1
            csv_writer.writerow(working_df)
                
        sch = spark.sql("select * from " + i[1]).schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch1_" + i[1] +".csv", sep=',')
        #df.write.mode("append").insertInto(i[1])
                
    elif i[1] == "DimCustomer":
        csv_file = open(os.getcwd() + "/result/Batch1_" + i[1] +".csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)


        df = pd.read_csv("CustomerMgmt.csv", header=None,sep=r'[,|]')
        
        serrogate_key_Customer = 0

        for index, row in df.iterrows():
            if row[0] == 'NEW' or row[0] == 'UPDCUST':
                Status = 'ACTIVE'
            else:
                Status = 'INACTIVE'
            if row[4] == 'F' or  row[4] == 'H':
                Gender = row[4]
            else: 
                Gender ='U'
            Phone1 = str(row[18])+str(row[19])+str(row[20])+str(row[21])
            Phone2 = str(row[22])+str(row[23])+str(row[24])+str(row[25])
            Phone3 = str(row[26])+str(row[27])+str(row[28])+str(row[29])

            NationalTax = spark.sql("select TX_NAME, TX_RATE from TaxRate where TX_ID ='" +str(row[31]) + "'")
            NationalTaxRateDesc = NationalTax.collect()[0][0]
            NationalTaxRate = NationalTax.collect()[0][1]

            LocalTax = spark.sql("select TX_NAME, TX_RATE from TaxRate where TX_ID = '" + str(row[30]) + "'")
            LocalTaxRateDesc = LocalTax.collect()[0][0]
            LocalTaxRate = LocalTax.collect()[0][1]

            working_df = [serrogate_key_Customer, row[2],row[3],Status, row[7],row[8],row[9],Gender,row[5],row[6],
                         row[10],row[11],row[12],row[13],row[14],row[15],Phone1,Phone2,Phone3,row[16],row[17]
                         ,NationalTaxRateDesc,NationalTaxRate,LocalTaxRateDesc,LocalTaxRate,
                          '','','','',1,1,row[1].split('T')[0],'9999-12-31']
            csv_writer.writerow(working_df)
            #raise Exception("Sorry, no numbers below zero") 
            serrogate_key_Customer+=1
            
            # DIMESSAGES VALERIO
            if row[5] not in [1,2,3]:
                df = [(current_timestamp(),1,"DimCustomer","Invalid customer tier","Alert",f"C_ID = {row[2]}, C_TIER ={row[5]}")]
                df.write.mode("append").insertInto("DImessages")
                
            if row[6] < datetime(1923,11,15) or row[6] > (2023,11,15):
                df = [(current_timestamp(),1,"DimCustomer","DOB out of range","Alert",f"C_ID = {row[2]}, C_DOB ={row[6]}")]
                df.write.mode("append").insertInto("DImessages")
            # ---
        
            
        sch = spark.sql("select * from " + i[1]).schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch1_" + i[1] +".csv", sep=',')
        #df.write.mode("append").insertInto(i[1])
                                        
                                        
    elif i[1] == "DimAccount":
        csv_file = open(os.getcwd() + "/result/Batch1_" + i[1] +".csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)

        df = pd.read_csv("CustomerMgmt.csv", header=None,sep=r'[,|]')

        serrogate_key_Account = 0
        for index, row in df.iterrows():
            action_type = row[0]
            if action_type in ['NEW', 'ADDACCT', 'UPDACCT']:
                Status = 'ACTIVE'
            else:
                Status = 'INACTIVE'

            effective_date = row[1].replace("T", " ")
            # Filter DimBroker to get the relevant BrokerID based on CA_B_ID and effective date
            SK_BrokerID = spark.sql("select SK_BrokerID from DimBroker where EffectiveDate <= '"+ effective_date 
                                    +"' and EndDate >= '"+ effective_date 
                                    +"' and BrokerID == '" + str(row[34]) + "'")#.collect()[0][0]

            # Filter DimCustomer to get the relevant CustomerID based on C_ID and effective date
            SK_CustomerID = spark.sql("select SK_CustomerID from DimCustomer where EffectiveDate <= '"+ effective_date 
                                    +"' and EndDate >= '"+ effective_date 
                                    +"' and CustomerID == '" + str(row[2]) + "'")#.collect()[0][0]

            working_df = [serrogate_key_Account, row[32], SK_BrokerID, SK_CustomerID, Status, row[35],
                                  row[33], True, 1, effective_date, '9999-12-31']
            
            csv_writer.writerow(working_df)
            serrogate_key_Account += 1
        sch = spark.sql("select * from " + i[1]).schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch1_" + i[1] +".csv", sep=',')
        #df.write.mode("append").insertInto(i[1])
        
    elif i[1] == "DimTrade":
        csv_file = open(os.getcwd() + "/result/Batch1_" + i[1] +".csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)

        trade_data = pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/Trade.txt", sep='|', header=None)
        trade_history_data = pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/TradeHistory.txt", sep='|', header=None)

        trade_data.columns = ['T_ID', 'TH_DTS', 'TH_ST_ID', 'T_TT_ID', 'T_IS_CASH', 'T_S_SYMB', 'T_QTY', 'T_BID_PRICE', 'T_CA_ID', 'T_EXEC_NAME', 'T_TRADE_PRICE', 'T_CHRG', 'T_COMM', 'T_TAX']
        trade_history_data.columns = ['T_ID', 'TH_DTS', 'TH_ST_ID']

        #trade_data['TH_DTS'] = pd.to_datetime(trade_data['TH_DTS'], format='%Y-%m-%d %H:%M:%S')
        #trade_history_data['TH_DTS'] = pd.to_datetime(trade_history_data['TH_DTS'], format='%Y-%m-%d %H:%M:%S')
        
        for index, row in trade_data.iterrows():
            time = spark.sql("select SK_DATEID from DimTime where SECONDDESC == '"+row[1].split(" ")[1] + "'").collect()[0][0]
            date = spark.sql("select SK_DATEID from DimDate where DATEVALUE == '"+row[1].split(" ")[0] + "'").collect()[0][0]
            
            
            if (row[2] == "SBMT" and (row[3] == "TMB" or row[3] == "TMS")) or row[2] == "PNDG":
                SK_CreateDateID = date
                SK_CreateTimeID = time
                SK_CloseDateID = ''
                SK_CloseTimeID = ''

            elif row[2] in ['CMPT', 'CNCL']:
                print(row[1])
                SK_CreateDateID = ''
                SK_CreateTimeID = ''
                SK_CloseDateID = date
                SK_CloseTimeID = time
            tmp = '' #spark.sql("select SK_AccountID, SK_CustomerID, SK_BrokerID from DimAccount where AccountID = '" 
                                  #       + row[?] + "' and "+row[1]+ "between EffectiveDate and EndDate" )
            SK_BrokerID = '' #tmp.collect()[0][0]
            SK_CustomerID = '' #tmp.collect()[0][1]
            SK_AccountID = '' #tmp.collect()[0][2]
            Status = '' #spark.sql("select ST_NAME from StatusType where ST_ID = '" + row[?]+ "'" )
            Type = '' #spark.sql("select TT_NAME from TradeType where TT_ID = '" + row[?]+ "'" )
            tmp = '' #spark.sql("select SK_SecurityID, SK_CompanyID from DimSecurity where SYMBOL = '" + row[1] + 
                               #          "' and EFFECTIVEDATE ="+row[0] +"'")
            SK_SecurityID = '' #tmp.collect()[0][0]
            SK_CompanyID = '' #tmp.collect()[0][1]
            
            
            #Pour tout ce qui est SK, ainsi que Status et Type y a juste à faire le match avec les autres tables et récupérer les valeurs
            working_df = [row[0], SK_BrokerID, SK_CreateDateID, SK_CreateTimeID, SK_CloseDateID,
                                          SK_CloseTimeID, Status, Type, row[4], SK_SecurityID, SK_CompanyID,
                                          row[6], row[7], SK_CustomerID, SK_AccountID, row[9], row[10], row[11],
                                          row[12], row[13], 1]
            csv_writer.writerow(working_df_trade)
            
            # DIMESSAGES VALERIO
            if row[17] is not None and row[17] > (row[16] * row[11]):
                df = [(current_timestamp(),1,"DimTrade","Invalid trade fee","Alert",f"T_ID = {row[0]}, T_CHRG ={row[17]}")]
                df.write.mode("append").insertInto("DImessages")
                
            if row[18] is not None and row[18] > (row[16] * row[11]):
                df = [(current_timestamp(),1,"DimTrade","Invalid trade commission","Alert",f"T_ID = {row[0]}, T_COMM ={row[18]}")]
                df.write.mode("append").insertInto("DImessages")
            # ---
                                        
                                        
        sch = spark.sql("select * from " + i[1]).schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch1_" + i[1] +".csv", sep=',')
        #df.write.mode("append").insertInto(i[1])

In [ ]:
FACT TABLE LOAD

In [ ]:

for i in Modified_FactTable:
    name_file = glob.glob(os.getcwd() + "/data/SF"+str(SF) +"/Batch1/" +i[0])
    print(i[0])
    if i[1] == "FactCashBalances" or False:
        csv_file = open(os.getcwd() + "/result/" + i[1] + ".csv", 'w', newline='')
        csv_writer = csv.writer(csv_file)
        

        df = pd.read_csv(name_file[0], header=None,sep=r'[,|]')
        for index, row in df.iterrows():
            working_df = []
            if i[1] == "FactCashBalances":
                #set value
                info_dim=spark.sql(f"""SELECT SK_CustomerID,SK_AccountID FROM DimAccount
                                        WHERE ACCOUNTID = {row[0]} AND EFFECTIVEDATE <= '{row[1].split(" ")[0]}'
                                        AND '{row[1].split(" ")[0]}' <= ENDDATE""")

                SK_CustomerID=info_dim.collect()[0][0]
                SK_AccountID=info_dim.collect()[0][1]
                SK_DateID = spark.sql(f"""SELECT SK_DateID FROM DimDate
                                        where DateValue = '{row[1].split(" ")[0]}' """).collect()[0][0]
                Cash = 0 # spark.sql("select SK_DateID from DimDate where DateValue = '" + row[1] + "'")
                BatchID =1

                working_df = [SK_CustomerID,SK_AccountID,SK_DateID,Cash,BatchID]
            elif i[1] == "FactMarketHistory":
                #set value
                tmp = '' #spark.sql("select SK_CustomerID, SK_CompanyID from DimSecurity where Symbol = '" + row[3] + " where IsCurrent = '1'")
                SK_SECURITYID = '' #tmp.collect()[0][0]
                SK_COMPANYID = '' #tmp.collect()[0][1]
                SK_DATEID= '' #spark.sql("select SK_DATEID from DimDate where DateValue = '" + row[0] + "'")
                PERATIO= '' # sprak ????
                YIELD= '' #spark.sql("select dividend from DimSecurity where SYMBOL = '" + row[1] + "'and "+row[0]+
                       #          "between EffectiveDate and EndDate")/row[2]
                FIFTYTWOWEEKHIGH = ''#spark.sql("select max(DM_HIGH) from FactMarketHistory 
                        #where" + row[0] +" <> date_sub("+row[0]+", 365))"
                FIFTYTWOWEEKLOW= '' #spark.sql("select min(DM_LOW) from FactMarketHistory 
                                #where" + row[0] +" <> date_sub("+row[0]+", 365))"
                SK_FIFTYTWOWEEKLOWDATE = '' # ??
                SK_FIFTYTWOWEEKHIGHDATE = '' #??                                                         #         " from tmp where" + row[0] +" <> date_sub("+row[0]+", 365))"

                CLOSEPRICE=row[2]
                DAYHIGH=row[3]
                DAYLOW=row[4]
                VOLUME=row[5]
                BATCHID=1

                working_df = [SK_SecurityID,SK_COMPANYID,SK_DATEID,PERATIO,YIELD,FIFTYTWOWEEKHIGH,
                              SK_FIFTYTWOWEEKHIGHDATE,FIFTYTWOWEEKLOW,SK_FIFTYTWOWEEKLOWDATE,
                              CLOSEPRICE,DAYHIGH,DAYLOW,VOLUME,BATCHID]
                
                # DIMESSAGES VALERIO
                if row[3] is None:
                     df = [(current_timestamp(),1,"FactMarketHistory","No earnings for company","Alert",f"DM_S_SYMB={row[3]}")]
                     df.write.mode("append").insertInto("DImessages")
                # ---
                
            elif i[1] == "FactWatches":
                #SK_CUSTOMERID = 0#spark.sql("select SK_CustomerID from DimCustomer where CUSTOMERID = '" + row[0] + "'")
                #SK_SECURITYID = 0#spark.sql("select SK_SecurityID from DimSecurity where SYMBOL = '" + row[1] + "'")
                
                SK_CUSTOMERID = spark.sql(f"""
                                    SELECT SK_CUSTOMERID
                                    FROM DimCustomer
                                    WHERE CUSTOMERID = {row[0]} """).collect()[0][0] 



                SK_SECURITYID = spark.sql(f"""
                                    SELECT SK_SecurityID
                                    FROM DimSecurity
                                    WHERE SYMBOL = '{row[1]}' """).collect()[0][0] 
                SK_DATEID_DATEPLACED=row[2]
                if row[3]=="ACTV":
                    SK_DATEID_DATEREMOVED = ''
                else:
                    SK_DATEID_DATEREMOVED = row[2]
                BATCHID=1

                working_df = [SK_CUSTOMERID,SK_SECURITYID,SK_DATEID_DATEPLACED,SK_DATEID_DATEREMOVED,BATCHID]


            elif i[1] == "FactHoldings":
                print(row)
                TRADEID = row[0]
                CURRENTTRADEID = row[1]
                df = spark.sql("""select SK_CustomerID,SK_AccountID,SK_SECURITYID,SK_COMPANYID,
                SK_CLOSEDATEID,SK_CloseTimeID, SK_AccountID
                               from DimTrade where TradeID = '""" + str(row[1]) + "'")
                
                
                SK_CUSTOMERID =df.collect()[0][0]
                SK_ACCOUNTID = df.collect()[0][1]
                SK_SECURITYID = df.collect()[0][2]
                SK_COMPANYID = df.collect()[0][3]
                SK_DATEID =df.collect()[0][4]
                SK_TIMEID = df.collect()[0][5]
                CURRENTPRICE =df.collect()[0][6]
                CURRENTHOLDING = row[3]
                BATCHID = 1                                                            
                working_df = [TRADEID,CURRENTTRADEID,SK_CUSTOMERID,SK_ACCOUNTID,SK_SECURITYID,SK_COMPANYID,
                              SK_DATEID,SK_TIMEID,CURRENTPRICE,CURRENTHOLDING,BATCHID]

            elif i[1] == "Prospect":
                isCustomer = spark.sql("select * from DimCustomer where status = 'ACTIVE' ")#to-do
                if isCustomer.count() == 0:
                    isCustomer = 0
                else:
                    isCustomer = 1
                SK_RecordDateID = 0 #to-do
                SK_UpdateDateID = 0 #to-do
                
                MarketingNameplate = ""
                if row[12] > 200000 or row[21] > 1000000:
                    MarketingNameplate += "+HighValue"
                if row[14] > 3 or row[20] > 5:
                    MarketingNameplate += "+Expenses"
                if row[16] > 45:
                    MarketingNameplate += "+Boomer"
                if row[12] < 100000 or row[21] < 50000 or row[17] < 600:
                    MarketingNameplate += "+MoneyAlert"
                if row[13] > 3 or row[20] > 7 :
                    MarketingNameplate += "+Spender"
                if row[16] < 25 and row[12] > 1000000 :
                    MarketingNameplate += "+Inherited"
                if len(MarketingNameplate) != 0:
                    MarketingNameplate = MarketingNameplate[1:]
                
                
                working_df = [row[0],SK_RecordDateID,SK_UpdateDateID,1,isCustomer
                              ,row[1],row[2],row[3],row[4],row[5],row[6]
                              ,row[7],row[8],row[9],row[10]
                             ,row[11],row[12],row[13],row[14],row[15],row[16],
                              row[17],row[18],row[19],row[20],row[21],MarketingNameplate]
                print(MarketingNameplate)

            csv_writer.writerow(working_df)

        #add result 
        sch = spark.sql("select * from " + i[1]).schema
        df = spark.read.schema(sch).csv(os.getcwd() + "/result/" + i[1] + ".csv", sep=',')
        ·#df.write.mode("append").insertInto(i[1])
        
        if i[1] == "Prospect":
            df = [(current_timestamp(),1,"Prospect","Inserted rows","Status",prospect_count)]
            df.write.mode("append").insertInto("DImessages")


In [11]:
# AUDIT VALERIO
# Execute the following line during the Historical Phase
audit_upload(1)

# At the end of the Historical Load: execute the batch validation query
#spark.sql(os.getcwd() + "/data/output/tpcdi_validation.sql")

f = open(os.getcwd() + "/ddl/tpcdi_validation.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

23/12/18 15:29:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2002Q2_audit.csv
23/12/18 15:29:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1984Q3_audit.csv
23/12/18 15:29:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2006Q1_audit.csv
23/12/18 15:30:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1990Q1_audit.csv
23/12/18 15:30:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1996Q2_audit.csv
23/12/18 15:30:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1978Q3_audit.csv
23/12/18 15:30:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1997Q4_audit.csv
23/12/18 15:30:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1992Q2_audit.csv
23/12/18 15:30:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1999Q3_audit.csv
23/12/18 15:30:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2003Q3_audit.csv
23/12/18 15:30:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2003Q2_audit.csv
23/12/18 15:30:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1973Q4_audit.csv
23/12/18 15:30:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2017Q1_audit.csv
23/12/18 15:30:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2006Q3_audit.csv
23/12/18 15:30:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE1985Q1_audit.csv
23/12/18 15:30:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2016Q1_audit.csv
23/12/18 15:30:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWa

23/12/18 15:30:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2015Q3_audit.csv
23/12/18 15:30:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/Prospect_audit.csv
23/12/18 15:30:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehou

23/12/18 15:30:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2013Q3_audit.csv
23/12/18 15:30:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: DataSet,  BatchID , Date ,  Attribute ,  Value,  DValue
 Schema: DATASET, BATCHID, DATE, ATTRIBUTE, VALUE, DVALUE
Expected: BATCHID but found:  BatchID 
CSV file: file:///home/coucou/ULB/MA2/DataWarehouse/Projet/TPC-DI/data/output/Batch1/FINWIRE2011Q3_audit.csv


In [12]:
spark.sql("select * from audit").show()

+-----------------+-------+----+----------+-------+------+
|          DATASET|BATCHID|DATE| ATTRIBUTE|  VALUE|DVALUE|
+-----------------+-------+----+----------+-------+------+
|      DimSecurity|      1|NULL|    FW_SEC|     19|  NULL|
|      DimSecurity|      1|NULL|FW_SEC_DUP|     -1|  NULL|
|       DimCompany|      1|NULL|    FW_CMP|     12|  NULL|
|       DimCompany|      1|NULL|FW_CMP_DUP|     -1|  NULL|
|        Financial|      1|NULL|    FW_FIN|    994|  NULL|
|        Financial|      1|NULL|FW_FIN_DUP|     -1|  NULL|
|FactMarketHistory|      1|NULL|DM_RECORDS|2422064|  NULL|
|      DimSecurity|      1|NULL|    FW_SEC|     19|  NULL|
|      DimSecurity|      1|NULL|FW_SEC_DUP|     -1|  NULL|
|       DimCompany|      1|NULL|    FW_CMP|     12|  NULL|
|       DimCompany|      1|NULL|FW_CMP_DUP|     -1|  NULL|
|        Financial|      1|NULL|    FW_FIN|   1849|  NULL|
|        Financial|      1|NULL|FW_FIN_DUP|     -1|  NULL|
|      DimSecurity|      1|NULL|    FW_SEC|     19|  NUL

## Incremental Load

In [13]:
# AUDIT VALERIO
# Run Visibility 1 at the start of incremental
#spark.sql(os.getcwd() + "/data/output/tpcdi_visibility_1.sql")

f = open(os.getcwd() + "/ddl/tpcdi_visibility_1.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

# Run Visibility 2 right after visibility 1
#spark.sql(os.getcwd() + "/data/output/tpcdi_visibility_2.sql")

f = open(os.getcwd() + "/tpcdi_visibility_2.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

# Register time
start_time = time.time()

In [ ]:
def incr_DimAccount(batch_id):
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_DimAccount.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
    
    account_data = pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/Account.txt", header=None,sep=r'[,|]')
    
    account_data.columns = ['I/U', 'CA_ID', 'CA_NAME', 'CA_TAX_ST', 'CA_B_ID', 'CA_C_ID', 'CA_ST_ID', 'CDC_FLAG']
    serrogate_key_Account = 0#spark.sql("select max(SK_ACCOUNTID) from DimAccount").collect()[0][0]
    print(account_data)
    for index, row in account_data.iterrows():
        if row[0] == "I":
            pass
        elif row[0] == "U":
            pass

        # Fetch SK_BrokerID and SK_CustomerID
        #matching_broker_record = DimBroker[DimBroker['BrokerID'] == row[4]]
        #matching_customer_record = DimCustomer[DimCustomer['CustomerID'] == row[5]]

        SK_BrokerID = matching_broker_record['SK_BrokerID'].values[0]
        SK_CustomerID = matching_customer_record['SK_CustomerID'].values[0]
        
        SK_BrokerID =0 #sprak.sql("select SK_BrokerID from DimBroker where BrokerID == '"+str(row[4])+"'").collect()[0][0]
        SK_CustomerID =0 #sprak.sql("select SK_CustomerID from DimCustomer where CustomerID == '"+str(row[5])+"'").collect()[0][0]

        Status = ''#sprak.sql("select ST_NAME from StatusType where ST_ID == '"+str(row[6])+"'").collect()[0][0]
        #StatusType[StatusType['ST_ID'] == row[6]]['ST_NAME'].values[0]


        working_df_account = [serrogate_key_Account, row[1], SK_BrokerID, SK_CustomerID, Status, row[2], row[3], 1, batch_id, '', '9999-12-31']
        #print(working_df_account)
        #raise Exception("Sorry, no numbers below zero") 
        csv_writer.writerow(working_df_account)
        raise ValueError('A very specific bad thing happened.')
        serrogate_key_Account += 1
        
    
    sch = spark.sql("select * from DimAccount").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_DimAccount.csv", sep=',')   
    #df.write.mode("append").insertInto("DimAccount")
    

def incr_DimCustomer(batch_id):
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_DimCustomer.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
    customer_data = pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/Customer.txt", header=None,sep=r'[,|]')
    serrogate_key_Customer = spark.sql("select max(SK_CUSTOMERID) from DimCustomer").collect()[0][0]
    for index, row in customer_data.iterrows():
        if row[0] == "I":
            pass
        elif row[0] == "U":
            pass
        if row[8] == 'F' or  row[8] == 'H':
            Gender = row[8]
        else: 
            Gender ='U'

        matching_status_record = StatusType[StatusType['ST_ID'] == row[4]]
        SK_StatusID = matching_status_record['SK_StatusID'].values[0]

        Phone1 = str(row[18])+str(row[19])+str(row[20])+str(row[21])
        Phone2 = str(row[22])+str(row[23])+str(row[24])+str(row[25])
        Phone3 = str(row[26])+str(row[27])+str(row[28])+str(row[29])

        NationalTax = spark.sql("select TX_NAME, TX_RATE from TaxRate where TX_ID ='" +str(row[31]) + "'")
        NationalTaxRateDesc = NationalTax.collect()[0][0]
        NationalTaxRate = NationalTax.collect()[1][0]

        LocalTax = spark.sql("select TX_NAME, TX_RATE from TaxRate where TX_ID = '" + str(row[30]) + "'")
        LocalTaxRateDesc = LocalTax.collect()[0][0]
        LocalTaxRate = LocalTax.collect()[1][0]

        working_df_customer = [
            serrogate_key_Customer, row[1], row[2], Status, row[5], row[6], row[7], Gender, row[9], row[10],
            row[11], row[12], row[13], row[14], row[15], Phone1,Phone2,Phone3,row[16],row[17], 
            NationalTaxRateDesc,NationalTaxRate,LocalTaxRateDesc,LocalTaxRate,
            '','','','', 1, batch_id, '', '9999-12-31'
        ]
        csv_writer.writerow(working_df_customer)
        serrogate_key_Customer += 1
        
        # VALERIO DIMESSAGES
        if row[5] not in [1,2,3]:
                df = [(current_timestamp(),batch_id,"DimCustomer","Invalid customer tier","Alert",f"C_ID = {row[2]}, C_TIER ={row[5]}")]
                df.write.mode("append").insertInto("DImessages")
                
            if row[6] < datetime(1923,11,15) or row[6] > (2023,11,15):
                df = [(current_timestamp(),batch_id,"DimCustomer","DOB out of range","Alert",f"C_ID = {row[2]}, C_DOB ={row[6]}")]
                df.write.mode("append").insertInto("DImessages")
        # ---
        
    #inserting result
    sch = spark.sql("select * from DimCustomer").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_DimCustomer.csv", sep=',')
    #df.write.mode("append").insertInto("DimCustomer")
    

def inc_FactCashBalance(batch_id):
    csv_file = open(os.getcwd() + "/result/Batch" + batch_id + "_FactCashBalance.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
    
    cash_transaction_data = pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/CashTransaction.txt", header=None,sep=r'[,|]')
    
    cash_transaction_data.columns = ['I/U', 'CT_ID', 'CT_CA_ID', 'CT_DATE', 'CT_AMT', 'CT_DESC']
    Cash = 0 # ??
    for index, row in cash_transaction_data.iterrows():
        if row['I/U'] == "I": # TO-DO
            pass
        elif row['I/U'] == "U":
            pass

        
        tmp = '' #spark.sql("select SK_CustomerID, SK_AccountID from DimAccount where AccountID = '" 
            #       + row[2] + "' and isCurrent == '1'" )
        SK_CustomerID ='' #tmp.collect()[0][0]
        SK_AccountID ='' #tmp.collect()[0][1]
        SK_DateID = ''#spark.sql("select SK_DateID from DimDate where Date == '"+ row[2]+"'").collect()[0][0]
        Cash = Cash + row[4]


        working_df_factCash = [SK_CustomerID, SK_AccountID, SK_DateID, Cash, batch_id]
        csv_writer.writerow(working_df_factCash)
        
    #inserting result
    sch = spark.sql("select * from FactCashBalance").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactCashBalance.csv", sep=',')
    #df.write.mode("append").insertInto("FactCashBalance")

        
def inc_DimTrade(batch_id): 
    
    csv_file = open(os.getcwd() + "/result/Batch" + batch_id + "_DimTrade.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
        
    trade_data=  pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/Trade.txt", header=None,sep=r'[,|]')
    trade_data.columns = ['CDC_FLAG', 'T_ID', 'T_CA_ID', 'T_DTS', 'T_ST_ID', 'T_TT_ID', 'T_IS_CASH', 'T_S_SYMB', 
                          'T_QTY', 'T_BID_PRICE', 'T_CA_ID_B', 'T_CA_ID_S', 'T_TRADE_PRICE', 'T_CHRG', 'T_COMM', 'T_TAX']

    for index, row in trade_data.iterrows():
        print(row[3])
        SK_CreateDateID = row[3].split(" ")[0]
        SK_CreateTimeID = row[3].split(" ")[1]
        SK_CloseDateID = None
        SK_CloseTimeID = None
        if row[4] in ["CMPT", "CNCL"]:
            SK_CloseDateID = row[3].split(" ")[0]
            SK_CloseTimeID = row[3].split(" ")[1]
        tmp = '' #spark.sql("select SK_AccountID, SK_CustomerID, SK_BrokerID from DimAccount where AccountID = '" 
                                  #       + row[?] + "' and IsCurrent = '1'"
        SK_BrokerID = '' #tmp.collect()[0][0]
        SK_CustomerID = '' #tmp.collect()[0][1]
        SK_AccountID = '' #tmp.collect()[0][2]
        Status = '' #spark.sql("select ST_NAME from StatusType where ST_ID = '" + row[?]+ "'" )
        Type = '' #spark.sql("select TT_NAME from TradeType where TT_ID = '" + row[?]+ "'" )
        tmp = '' #spark.sql("select SK_SecurityID, SK_CompanyID from DimSecurity where SYMBOL = '" + row[1] + 
                           #          "' and IsCurrent = '1'")
        SK_SecurityID = '' #tmp.collect()[0][0]
        SK_CompanyID = '' #tmp.collect()[0][1]

        working_df_trade = [row[1], SK_BrokerID, SK_CreateDateID, SK_CreateTimeID, SK_CloseDateID,
                                SK_CloseTimeID, Status, Type, row[6], SK_SecurityID, SK_CompanyID,
                                row[8], row[9], SK_CustomerID, SK_AccountID, row[11], row[12], row[13],
                                row[14], row[15], batch_id]
        csv_writer.writerow(working_df_trade)
        
        # DIMESSAGES VALERIO
        if row[17] is not None and row[17] > (row[16] * row[11]):
            df = [(current_timestamp(),batch_id,"DimTrade","Invalid trade fee","Alert",f"T_ID = {row[0]}, T_CHRG ={row[17]}")]
            df.write.mode("append").insertInto("DImessages")
                
        if row[18] is not None and row[18] > (row[16] * row[11]):
            df = [(current_timestamp(),batch_id,"DimTrade","Invalid trade commission","Alert",f"T_ID = {row[0]}, T_COMM ={row[18]}")]
            df.write.mode("append").insertInto("DImessages")
        # ---
        
        
    #inserting result
    sch = spark.sql("select * from DimTrade").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_DimTrade.csv", sep=',')
    #df.write.mode("append").insertInto("DimTrade")

incr_DimAccount(2)
    

In [ ]:
def inc_FactHoldings(batch_id): 
    
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactHoldings.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
        
    data=  pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/HoldingHistory.txt", header=None,sep=r'[,|]')
    
    for index, row in data.iterrows():
        TRADEID = row[2]
        CURRENTTRADEID = row[3]
        CURRENTHOLDING = row[5]
        
        df = spark.sql("""select SK_CustomerID,SK_AccountID,SK_SECURITYID,SK_COMPANYID,
                        SK_CLOSEDATEID,SK_CloseTimeID, SK_AccountID
                               from DimTrade where TradeID = '""" + str(row[3]) + "'")
        SK_CUSTOMERID =0 #df.collect()[0][0]
        SK_ACCOUNTID =0 #df.collect()[0][1]
        SK_SECURITYID =0 #df.collect()[0][2]
        SK_COMPANYID =0 #df.collect()[0][3]
        SK_DATEID =0 #df.collect()[0][4]
        SK_TIMEID =0 #df.collect()[0][5]
        CURRENTPRICE =0 #df.collect()[0][6]

        working_df = [TRADEID, CURRENTTRADEID, SK_CUSTOMERID, SK_ACCOUNTID, SK_SECURITYID, SK_COMPANYID,
                       SK_DATEID, SK_TIMEID, CURRENTPRICE, CURRENTHOLDING, batch_id]
        csv_writer.writerow(working_df)

    #inserting result
    sch = spark.sql("select * from FactHoldings").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactHoldings.csv", sep=',')
    #df.write.mode("append").insertInto("FactHoldings")
    
    
def inc_FactMarketHistory(batch_id): 
    
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactMarketHistory.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
        
    data=  pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/DailyMarket.txt", header=None,sep=r'[,|]')
    
    for index, row in data.iterrows():
        tmp = '' #spark.sql("select SK_CustomerID, SK_CompanyID from DimSecurity where Symbol = '" + row[3] + " where IsCurrent = '1'")
        SK_SECURITYID = '' #tmp.collect()[0][0]
        SK_COMPANYID = '' #tmp.collect()[0][1]
        SK_DATEID = '' #spark.sql("select SK_DateID from DimDate where DateValue = '" + row[2] + "'")
        PERATIO = ''
        YIELD = '' #spark.sql("select dividend from DimSecurity where Symbol = '" + row[3] + " where IsCurrent = '1'")/row[4]*100
        
        
        FIFTYTWOWEEKHIGH = ''#spark.sql("select max(DM_HIGH) from FactMarketHistory 
                        #where" + row[0] +" <> date_sub("+row[0]+", 365))"
        FIFTYTWOWEEKLOW= '' #spark.sql("select min(DM_LOW) from FactMarketHistory 
                        #where" + row[0] +" <> date_sub("+row[0]+", 365))"
        SK_FIFTYTWOWEEKLOWDATE = '' # ??
        SK_FIFTYTWOWEEKHIGHDATE = '' #??
        CLOSEPRICE = row[4] 
        DAYHIGH = row[5]
        DAYLOW = row[6]
        VOLUME = row[7]

        working_df = [SK_SECURITYID,SK_COMPANYID,SK_DATEID,PERATIO,YIELD,FIFTYTWOWEEKHIGH,
                              SK_FIFTYTWOWEEKHIGHDATE,FIFTYTWOWEEKLOW,SK_FIFTYTWOWEEKLOWDATE,
                              CLOSEPRICE,DAYHIGH,DAYLOW,VOLUME,batch_id]
        csv_writer.writerow(working_df)
     
        if row[3] is None:
            df = [(current_timestamp(),batch_id,"FactMarketHistory","No earnings for company","Alert",f"DM_S_SYMB={row[3]}")]
            df.write.mode("append").insertInto("DImessages")   
        
        
    #inserting result
    sch = spark.sql("select * from FactMarketHistory").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactMarketHistory.csv", sep=',')
    #df.write.mode("append").insertInto("FactMarketHistory")
    
def inc_FactWatches(batch_id): 
    
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactWatches.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
        
    data=  pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/WatchHistory.txt", header=None,sep=r'[,|]')
    
    for index, row in data.iterrows():
        # SK_CUSTOMERID and SK_SECURITYID: slighly different (added "IsCurrent=1").
        # Put in as a comment as this is how it was done in FactWatches's historical load
        SK_CUSTOMERID = 0 #spark.sql("select SK_CustomerID from DimCustomer where CUSTOMERID = '" + row[0] + "'AND IsCurrent = 1")
        SK_SECURITYID = 0 #spark.sql("select SK_SecurityID from DimSecurity where SYMBOL = '" + row[1] + "'AND IsCurrent = 1")
        SK_DATEID_DATEPLACED=row[2]
        if row[3]=="ACTV":
            SK_DATEID_DATEREMOVED = ''
        else:
            SK_DATEID_DATEREMOVED = row[2]
        

        working_df = [SK_CUSTOMERID,SK_SECURITYID,SK_DATEID_DATEPLACED,SK_DATEID_DATEREMOVED,batch_id]
        csv_writer.writerow(working_df)
        
    #inserting result
    sch = spark.sql("select * from FactWatches").schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_FactWatches.csv", sep=',')
    #df.write.mode("append").insertInto("FactWatches")
    
    
    
def inc_Prospect(batch_id):
    
    prospect_count = 0 
    
    # PERFORM UPDATE VALERIO
    spark.sql("CREATE TABLE temp_Prospect AS SELECT * FROM Prospect")
    
    csv_file = open(os.getcwd() + "/result/Batch" + str(batch_id) + "_Prospect.csv", 'w', newline='')
    csv_writer = csv.writer(csv_file)
        
    data=  pd.read_csv(os.getcwd() + "/data/SF"+str(SF) +"/Batch"+str(batch_id) +"/Prospect.csv", header=None,sep=r'[,|]')
    
    for index, row in data.iterrows():
        
        # DIMESSAGES VALERIO
        prospect_count += 1
        
        isCustomer = spark.sql("select * from DimCustomer where status = 'ACTIVE' ")#to-do
        if isCustomer.count() == 0:
            isCustomer = 0
        else:
            isCustomer = 1
        SK_RecordDateID = 0 #to-do (both historical and incremental)
        SK_UpdateDateID = 0 #to-do (both historical and incremental)
        
        MarketingNameplate = ""
        if row[12] > 200000 or row[21] > 1000000:
            MarketingNameplate += "+HighValue"
        if row[14] > 3 or row[20] > 5:
            MarketingNameplate += "+Expenses"
        if row[16] > 45:
            MarketingNameplate += "+Boomer"
        if row[12] < 100000 or row[21] < 50000 or row[17] < 600:
            MarketingNameplate += "+MoneyAlert"
        if row[13] > 3 or row[20] > 7 :
            MarketingNameplate += "+Spender"
        if row[16] < 25 and row[12] > 1000000 :
            MarketingNameplate += "+Inherited"
        if len(MarketingNameplate) != 0:
            MarketingNameplate = MarketingNameplate[1:]
        
        
        working_df = [row[0],SK_RecordDateID,SK_UpdateDateID,1,isCustomer
                    ,row[1],row[2],row[3],row[4],row[5],row[6]
                    ,row[7],row[8],row[9],row[10]
                    ,row[11],row[12],row[13],row[14],row[15],row[16],
                    row[17],row[18],row[19],row[20],row[21],MarketingNameplate]
        csv_writer.writerow(working_df)
    
    # PERFORM UPDATE VALERIO
    sch = spark.sql("select * from " + i[1]).schema
    df = spark.read.schema(sch).csv(os.getcwd() + "/result/temp_" + i[1] + ".csv", sep=',')
    if i[1] != "Prospect"
        #df.write.mode("append").insertInto(i[1])
    else:
        # Insert batch2's Prospect table values into the temporary table
        df.write.mode("append").insertInto("temp_Prospect")
        # Extract batch1's values not present in batch2
        input_query = """
        SELECT *
        FROM Prospect as p
        WHERE p.AgencyID NOT IN
        (
            SELECT tp.AgencyID
            FROM temp_Prospect as tp
        );
        """
        # Append them to the temporary table, which now contains "the updated"
        # values and the non-modified ones from Batch1
        spark.sql(input_query).write.insertInto("temp_Prospect", overwrite=False)
        # Erase all Prospect rows and copy temp_Prospect into it
        spark.sql("TRUNCATE TABLE Prospect")
        spark.sql("INSERT INTO Prospect SELECT * FROM temp_Prospect")
        # Erase all temp_Prospect rows
        spark.sql("TRUNCATE TABLE temp_Prospect")
    
    # DIMESSAGES VALERIO
    df = [(current_timestamp(),batch_id,"Prospect","Source rows","Status",prospect_count)]
    df.write.mode("append").insertInto("DImessages")
    df = [(current_timestamp()batch_id,"Prospect","Inserted rows","Status",123)]
    df.write.mode("append").insertInto("DImessages")
    df = [(current_timestamp(),batch_id,"Prospect","Updated rows","Status",123)]
    df.write.mode("append").insertInto("DImessages")
        
    # VERSION WITHOUT PERFORM UPDATE VALERIO
    #inserting result
    #sch = spark.sql("select * from Prospect").schema
    #df = spark.read.schema(sch).csv(os.getcwd() + "/result/Batch" + str(batch_id) + "_Prospect.csv", sep=',')
    #df.write.mode("append").insertInto("Prospect")


In [ ]:
# AUDIT VALERIO
for i in range(2,4,1):
    #incremental part
    # VERIFY ORDER !!!!!!!
    incr_DimAccount(i)
    visibility_2_fun() 
    incr_DimCustomer(i)
    visibility_2_fun()
    inc_FactCashBalance(i)
    visibility_2_fun()
    inc_DimTrade(i)
    visibility_2_fun()
    inc_FactHoldings(i)
    visibility_2_fun()
    inc_FactMarketHistory(i)
    visibility_2_fun()
    inc_FactWatches(i)
    visibility_2_fun()
    inc_Prospect(i)
    visibility_2_fun()
    #test part

In [ ]:
# AUDIT VALERIO
# Execute the following line during the Incremental Phase 1
audit_upload(2)
# At the end of the Incremental Phase 1: execute the batch validation query
#spark.sql(os.getcwd() + "/data/output/tpcdi_validation.sql")

f = open(os.getcwd() + "/tpcdi_validation.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

In [ ]:
# AUDIT VALERIO
# Execute the following line during the Incremental Phase 2
audit_upload(3)
# At the end of the Incremental Phase 2: execute the batch validation query
#spark.sql(os.getcwd() + "/data/output/tpcdi_validation.sql")

f = open(os.getcwd() + "/tpcdi_validation.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

# Automated Audit Phase

In [ ]:
# AUDIT VALERIO
# The following line execute the data visibility query 1. It has to be executed once at the start of the Automated Audit Phase
#spark.sql(os.getcwd() + "/data/output/tpcdi_visibility_1.sql")

f = open(os.getcwd() + "/tpcdi_visibility_1.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

# Then the audit query has to be runned once
#spark.sql(os.getcwd() + "/data/output/tpcdi_audit.sql")

f = open(os.getcwd() + "/tpcdi_audit.sql", "r")
tables=f.read()
tables = tables.split(";")[:-1]
for i in tables:
    spark.sql(i)

In [ ]:
# AUDIT VALERIO
# At the start of the audit phase, erase "temp_Prospect"
spark.sql("DROP TABLE IF EXISTS temp_Prospect")

# After Automated Audit (metric computation)

In [ ]:
# In TPC-DI the times are given as output of the Batch Validation query,
# so we need to retreive them from DImessages table

# CT = Completion Timestamp
CT0 = spark.sql(
    "select MessageDateAndTime from DImessages where BatchID = 0 and MessageType = ‘PCR’"
)
CT1 = spark.sql(
    "select MessageDateAndTime from DImessages where BatchID = 1 and MessageType = ‘PCR’"
)
CT2 = spark.sql(
    "select MessageDateAndTime from DImessages where BatchID = 2 and MessageType = ‘PCR’"
)
CT3 = spark.sql(
    "select MessageDateAndTime from DImessages where BatchID = 3 and MessageType = ‘PCR’"
)

# TH = Throughput of Historical Load
# TH = RH / EH
# --> RH = Row count of Batch1 (from "digen_report.txt")
# --> EH = Elapsed time for Historical Load
TH = 7804509 / (CT1 - CT0)

# For incremental phases, similar to what did before, except that
# the denominator is the maximum betweeen the elapsed time and 1800
TI1 = 33380 / max(CT2 - CT1, 1800)
TI2 = 33455 / max(CT3 - CT2, 1800)

TPC_DI_RPS = round(gmean([TH, min(TI1, TI2)]))